In [2]:
import nest_asyncio
from dotenv import load_dotenv
import os
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore


# Load Neo4j credentials
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

nest_asyncio.apply()
# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    url=NEO4J_URI,
)

In [1]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/").load_data()

In [3]:
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0)
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

c:\Saravana\My Github Repo\clinical-neo4j\.cvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]


In [10]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("What is transplant kidney infection?")

for node in nodes:
    print(node.text)

Osijek University Hospital -> PART_OF -> nan
Danderyd Hospital, Karolinska Institutet -> PART_OF -> Karolinska University Hospital
NCT06186947 -> PART_OF -> The Affiliated Hospital of Qingdao University
University Hospitals Dorset NHS Foundation Trust -> PART_OF -> Kaiser Permanente
University Hospitals Dorset NHS Foundation Trust -> PART_OF -> University of Witwatersrand


In [12]:
query_engine = index.as_query_engine(include_text=True)
query = "what symptoms with corona and tell me the sites, list nct numbers too"
query = "What is transplant kidney infection? which sites can help with that"
response = query_engine.query(query)

print(str(response))

Transplant kidney infection refers to an infection that occurs in the kidney of a transplant recipient. Sites that can help with transplant kidney infections include hospitals with specialized transplant units, such as University Hospital Kiel, University Hospital Bonn, Johannes Gutenberg University Mainz, University Hospital Essen, and Klinikum rechts der Isar der TUM.
